<a href="https://colab.research.google.com/github/elimeyer1/ML_4105/blob/main/Homework_6_Question_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
input_size = 3 * 32 * 32
hidden_sizes = [256, 128, 64]
num_classes = 10
num_epochs = 100
batch_size = 64
learning_rate = 0.001

In [11]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [12]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
class MultiLayerNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(MultiLayerNetwork, self).__init__()


        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, num_classes))

        self.network = nn.Sequential(*layers)

    def forward(self, x):

        x = x.view(x.size(0), -1)
        return self.network(x)

In [15]:
model = MultiLayerNetwork(input_size, hidden_sizes, num_classes).to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
train_losses = []
test_accuracies = []

In [18]:
def train_model():
    start_time = time.time()
    total_loss = 0

    for epoch in range(num_epochs):
        epoch_loss = 0
        for i, (images, labels) in enumerate(train_loader):

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()


        avg_loss = epoch_loss / len(train_loader)
        total_loss += avg_loss

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    training_time = time.time() - start_time
    avg_total_loss = total_loss / num_epochs

    return training_time, avg_total_loss

In [19]:
def evaluate_model():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

In [20]:
training_time, avg_loss = train_model()
accuracy = evaluate_model()

Epoch [10/100], Loss: 0.8787
Epoch [20/100], Loss: 0.5133
Epoch [30/100], Loss: 0.3503
Epoch [40/100], Loss: 0.2654
Epoch [50/100], Loss: 0.1985
Epoch [60/100], Loss: 0.1806
Epoch [70/100], Loss: 0.1477
Epoch [80/100], Loss: 0.1512
Epoch [90/100], Loss: 0.1378
Epoch [100/100], Loss: 0.1211


In [21]:
print("\n--- Results Report ---")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Training Loss: {avg_loss:.4f}")
print(f"Test Accuracy: {accuracy:.2f}%")


--- Results Report ---
Training Time: 2179.06 seconds
Average Training Loss: 0.3534
Test Accuracy: 51.48%
